## Stallion scraping notebook
***

Script to search racing post and get basic information about pedigree, stallion performance when racing

Then get detailed progeny statistics

Import packages

In [2]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time
import random
import decimal

***

Define output dataframe column headers

In [3]:
df1_cols = ['age',
           'sire',
           'dam',
           'dam_sire',
           'stud_fee',
           'total_starts',
           'total_earnings',
           'top_rpr',
           'sire_comments'
           ]

cols_df = []

info_to_take = ['Flat','Turf','All-weather','Jumps','Chase','Hurdles',
                'NHF','Broodmare sires','2yo','First Crop','5-6f','7-9f','10-11f',
                '12-13f','14f+','Heavy','Soft','Gd-sft','Good','Gd-fm','Firm']

for i in info_to_take:
    new1 = str(i) + "_win_runners"
    new2 = str(i) + "_wins"
    new3 = str(i) + "_runs"
    new4 = str(i) + "_earnings"
    
    cols_df.append(new1)
    cols_df.append(new2)
    cols_df.append(new3)
    cols_df.append(new4)

In [4]:
horse_list = pd.read_csv('../../input_data/stallion_list.csv', encoding= 'unicode_escape')
horse_list.info()


horse_ls = list(horse_list['STALLION NAME'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STALLION NAME  311 non-null    object 
 1   STANDING       310 non-null    object 
 2   CTRY           310 non-null    object 
 3   STUD FEE       310 non-null    object 
 4   SIRE           310 non-null    object 
 5   SIRE LINE      310 non-null    object 
 6   YEARÊTO STUD   310 non-null    float64
dtypes: float64(1), object(6)
memory usage: 17.5+ KB


Now write the functions to get the data, differenciating weatherby from non-weatherby registered stallions as they have different browser patterns
***

In [5]:
def find_the_correct_google_link(horse):
    url2='http://www.google.com'
    browser.get(url2)
        
    time.sleep(2)
    
    search = browser.find_element(By.NAME, 'q')
    search.send_keys(str(horse) + ' racing post progeny')
    search.send_keys(Keys.RETURN) # hit return after you enter search text
    
    time.sleep(2)
    
    try:
        search_form=browser.find_elements(By.XPATH, '//h3[@class="LC20lb DKV0Md"]')
        time.sleep(2)
        search_form[0].click()
    except:
        search_form=browser.find_elements(By.XPATH, '//h3[@class="LC20lb DKV0Md"]')
        time.sleep(2)
        search_form[4].click()
        
# browser.find_element(By.XPATH, '(//h3[@class="LC20lb"])[1]/a').click()

In [13]:
def weatherby(horse):
        
    search_form=browser.find_elements(By.XPATH, '//a[@class="ui-tabs__tab pp-tabs__tab hidden-xs-down hidden-sm-down"]')
    search_form[0].click()
        
    time.sleep(3)
        
    timeout = 10
    WebDriverWait(browser, timeout).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='hp-raceRecords__label']")))
        
    element2 = browser.find_elements(By.XPATH, "//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]
    year_perf=titles[-36:]
    year_perf_2 = titles[-18:]
    top_rpr = titles[10]
    total_earnings = titles[7]
    total_starts=titles[1]

    age_element = browser.find_elements(By.XPATH, "//dt[@class='pp-definition__term']")
    age_element_info = [x.text for x in age_element]
    age=age_element_info[0]

    sire = browser.find_elements(By.XPATH, "//a[@class='ui-link ui-link_table js-popupLink hp-horseDefinition__link']")
    sire_info = [x.text for x in sire]

    stud = browser.find_elements(By.XPATH, "//span[@class='hp-nameRow__studFee__value']")
    stud_fee = [x.text for x in stud]
    
    comments = browser.find_elements(By.XPATH, "//dd[@class='pp-definition__description']")
    sire_comments = [x.text for x in comments]

    table_option1 = [age,
                    sire_info[0],
                    sire_info[1],
                    sire_info[2],
                    stud_fee[0],
                    total_starts,
                    total_earnings,
                    top_rpr,
                    sire_comments[5]]

    df1.loc[horse,:]=table_option1
    
    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

    search_form=browser.find_elements(By.XPATH, '//a[@class="ui-tabs__tab pp-tabs__tab"]')
    search_form[1].click()

    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

    search_form=browser.find_elements(By.XPATH, '//a[@class="ui-link ui-link_primary ui-popoverList__link pp-subTabs__tab"]')
    search_form[0].click()

    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))
    
    element2 = browser.find_elements(By.XPATH, "//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]


    info_to_take = ['Flat','Turf','All-weather','Jumps','Chase','Hurdles',
                    'NHF','Broodmare sires','2yo','First Crop','5-6f','7-9f','10-11f',
                    '12-13f','14f+','Heavy','Soft','Gd-sft','Good','Gd-fm','Firm']

    time.sleep(float(decimal.Decimal(random.randrange(150, 300))/100))
    
    for i in info_to_take:
        if i in titles:
            col1 = str(i) + "_win_runners"
            col2 = str(i) + "_wins"
            col3 = str(i) + "_runs"
            col4 = str(i) + "_earnings"

            find_index = titles.index(i)

            win_runners = titles[find_index + 1]
            wins = titles[find_index + 3]
            runs = titles[find_index + 4]
            earnings = titles[find_index + 8]

            df2.loc[horse,[col1, col2, col3, col4]]=[win_runners, wins, runs, earnings]




    return df1, df2


In [7]:
def non_weatherby(horse):
    other_click = browser.find_elements(By.XPATH, '//button[@class="ui-reset-for-btn ui-modal__closeBtn"]')
    other_click[0].click()

    time.sleep(2)

    search_form=browser.find_elements(By.XPATH, '//a[@class="ui-tabs__tab pp-tabs__tab hidden-xs-down"]')
    search_form[0].click()

    timeout = 10
    WebDriverWait(browser, timeout).until(EC.visibility_of_element_located((By.XPATH, "//h2[@class='hp-formTable__title']")))


    element2 = browser.find_elements(By.XPATH, "//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]
    top_rpr = titles[10]
    total_earnings = titles[7]
    total_starts=titles[1]

    age_element = browser.find_elements(By.XPATH, "//dt[@class='pp-definition__term']")
    age_element_info = [x.text for x in age_element]
    age=age_element_info[0]

    sire = browser.find_elements(By.XPATH, "//a[@class='ui-link ui-link_table js-popupLink hp-horseDefinition__link']")
    sire_info = [x.text for x in sire]

    stud = browser.find_elements(By.XPATH, "//span[@class='hp-nameRow__studFee__value']")
    if not stud:
        stud_fee = 'not there'
    else:
        stud_fee = [x.text for x in stud]

    comments = browser.find_elements(By.XPATH, "//dd[@class='pp-definition__description']")
    sire_comments = [x.text for x in comments]

    table_option = [age,
                    sire_info[0],
                    sire_info[1],
                    sire_info[2],
                    stud_fee[0],
                    total_starts,
                    total_earnings,
                    top_rpr,
                    sire_comments[5]]


    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

    search_form=browser.find_elements(By.XPATH, '//a[@class="ui-tabs__tab pp-tabs__tab"]')
    search_form[0].click()

    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

    search_form=browser.find_elements(By.XPATH, '//a[@class="ui-link ui-link_primary ui-popoverList__link pp-subTabs__tab"]')
    search_form[0].click()

    time.sleep(float(decimal.Decimal(random.randrange(100, 150))/100))

    element2 = browser.find_elements(By.XPATH, "//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]

    time.sleep(float(decimal.Decimal(random.randrange(150, 300))/100))

    info_to_take = ['Flat','Turf','All-weather','Jumps','Chase','Hurdles',
                    'NHF','Broodmare sires','2yo','First Crop','5-6f','7-9f','10-11f',
                    '12-13f','14f+','Heavy','Soft','Gd-sft','Good','Gd-fm','Firm']


    for i in info_to_take:
        if i in titles:
            col1 = str(i) + "_win_runners"
            col2 = str(i) + "_wins"
            col3 = str(i) + "_runs"
            col4 = str(i) + "_earnings"

            find_index = titles.index(i)

            win_runners = titles[find_index + 1]
            wins = titles[find_index + 3]
            runs = titles[find_index + 4]
            earnings = titles[find_index + 8]

            df2.loc[horse,[col1, col2, col3, col4]]=[win_runners, wins, runs, earnings]




    return df1, df2

In [10]:
def racingpost(horse):
    
    time.sleep(3)
    
    find_the_correct_google_link(horse)
    
    time.sleep(3)
    
    timeout = 10
    try:
        weatherby(horse)
    except:
        non_weatherby(horse)
        

***

***

In [17]:
failed = []

browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('http://www.racingpost.com')

time.sleep(2)
browser.get('http://www.google.com')
time.sleep(1)

google_popup1 = browser.find_elements(By.XPATH, '//div[@class="jyfHyd"]')
google_popup2 = browser.find_elements(By.XPATH, '//div[@class="QS5gu sy4vM"]')  

if len(google_popup1) > 0:
    google_popup1[1].click()


if len(google_popup2) > 0:
    google_popup2[1].click()

 
df1=pd.DataFrame(columns=df1_cols)
df2=pd.DataFrame(columns=cols_df)

i=0

for horse in horse_ls:
    try:
        racingpost(horse)
    except:
        failed.append(horse)
    i=i+1
    print('{}/{}, failed={}'.format(i, len(horse_ls), len(failed)))


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/jackpickard/.wdm/drivers/chromedriver/mac64/95.0.4638.69/chromedriver] found in cache
/var/folders/jf/wwr3jnbx3zvfghjk_j_tbxnm0000gp/T/ipykernel_75830/1608919976.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


1/317, failed=0
2/317, failed=0
3/317, failed=0
4/317, failed=0
5/317, failed=0
6/317, failed=0
7/317, failed=0
8/317, failed=0
9/317, failed=0
10/317, failed=0
11/317, failed=0


***
Download output dataframes

In [ ]:
df1.to_csv('df1_11may20_stallions.csv')

In [ ]:
df2.to_csv('df2_11may20_stallions.csv')

In [ ]:
df3 = df2.merge(df1, left_index=True, right_index=True)

In [ ]:
df3.to_csv('progeny_stats_11may20_original.csv')

In [ ]:
df3.sample().transpose()